# Volcado Tabular de Vencimientos Explained con Otros Campos

In [1]:
from datetime import date
import pandas as pd
import requests

## Rutas

In [2]:
rutas = {
    "explained": "https://ms-tesoreria-analytics-staging.interconecta2.cl/v1/settlements/swaps/explained",
    "headers": "https://ms-tesoreria-analytics-staging.interconecta2.cl/operations/all",
}

## SDK

Estas son funciones que utilizan los endpoints adecuados para obtener la data necesaria para el volcado.

In [3]:
def get_explained(fecha: date, ruta: str):
    result = requests.get(f"{ruta}/{fecha}")
    if result.status_code == 200:
        return result.json()
    else:
        return result.status_code

In [4]:
def get_headers(ruta: str):
    result = requests.get(ruta)
    if result.status_code == 200:
        return result.json()
    else:
        return result.status_code

## Procedimiento

Se define la fecha de proceso.

In [5]:
fecha_proceso = date(2025, 8, 11)
f_proc = fecha_proceso.isoformat().replace("-", "")

Se obtienen los explained a fecha de proceso.

In [6]:
explained = get_explained(fecha_proceso, rutas["explained"])

Se obtienen los encabezados de la cartera vigente.

In [7]:
headers = get_headers(rutas["headers"])

Vemos un registro de los explained.

In [8]:
explained[0]

{'dealNumber': '1634',
 'legNumber': 1,
 'tipoPata': 'FIXED_RATE_MCCY',
 'recPay': 'A',
 'fechaInicial': '2025-02-10',
 'fechaFinal': '2025-08-11',
 'fechaPago': '2025-08-11',
 'nocional': 575000.0,
 'amortizacion': 0.0,
 'interes': 3691.8194444445,
 'amortEsFlujo': True,
 'flujo': 3691.8194444445,
 'monedaNocional': 'CLF',
 'valorTasa': 0.0127,
 'tipoTasa': 'LinAct360',
 'fechaFixingFx': '2025-08-11',
 'monedaPago': 'CLP',
 'indiceFx': 'UF',
 'valorIndiceFx': 39156.08,
 'amortizacionMonedaPago': 0.0,
 'interesMonedaPago': 144557177.5122244,
 'flujoMonedaPago': 144557177.5122244,
 'fraccionAgno': '(fechaFinal - fechaInicial) / 360',
 'formulaInteres': 'nocional * valorTasa * fraccionAgno',
 'formulaInteresMonedaPago': 'formulaInteres * valorIndiceFx',
 'formulaAmortizacionMonedaPago': 'amortizacion * valorIndiceFx'}

Se observa que no tiene los campos de portfolio (gestión o contable) ni la estrategia contable. Se los agregamos con la data de los headers.

In [9]:
df_explained = pd.DataFrame.from_dict(explained)

Vemos la estructura de un header.

In [10]:
headers["5000"]

{'trade_date': {'fecha': '2022-11-29'},
 'deal_number': '5000',
 'counterparty': {'name': 'BANCO SANTANDER-SANTIAGO',
  'rut': {'rut': 97036000, 'dv': 'K'},
  'other': None},
 'portfolio': 'BANCA',
 'hedge_accounting': 'COB_TASA_INT_ACTIVOS',
 'product': 'SWAP_MONE',
 'currency_pair': 'CLFCLP',
 'price': None,
 'settlement_mechanism': 'C',
 'other': {'regulatory_portfolio': 'COBERTURA_VALOR_RAZONABLE'}}

In [11]:
def generate_new_columns(row):
    # Notar que headers se toma por closure
    operacion = headers.get(row.dealNumber)
    if operacion is None:
        operacion = {"dummy": 1}
    if operacion.get("other") is None:
        regulatory_portfolio = None
    else:
        regulatory_portfolio = operacion.get("other").get("regulatory_portfolio")
    return pd.Series({
        'portfolio': operacion.get("portfolio"),
        'hedgeAccounting': operacion.get("hedge_accounting"),
        'product': operacion.get("product"),
        'settlementMechanism': operacion.get("settlement_mechanism"),
        'regulatoryPortfolio': regulatory_portfolio,
    })

In [12]:
df_explained_ok = df_explained.join(df_explained.apply(
    generate_new_columns,
    axis=1,
))
del df_explained_ok["fraccionAgno"]
del df_explained_ok["formulaValorTasaEquivalente"]
del df_explained_ok["formulaInteres"]
# TRY - EXCEPT
del df_explained_ok["formulaInteresMonedaPago"]
del df_explained_ok["formulaAmortizacionMonedaPago"]

In [13]:
df_explained_ok

,dealNumber,legNumber,tipoPata,recPay,fechaInicial,fechaFinal,fechaPago,nocional,amortizacion,interes,...,nombreIndice,valorIndiceInicial,valorIndiceFinal,numDecimalesTasaEquivalente,codigoIndiceTasa,portfolio,hedgeAccounting,product,settlementMechanism,regulatoryPortfolio
0,1634,1,FIXED_RATE_MCCY,A,2025-02-10,2025-08-11,2025-08-11,5.750000e+05,0.00,3.691819e+03,...,NaN,NaN,NaN,NaN,NaN,NEGOCIACION,NO,SWAP_ICP,C,NEGOCIACION
1,1634,2,ICP_CLF,P,2025-02-10,2025-08-11,2025-08-11,-5.750000e+05,0.00,-2.948804e+03,...,NaN,NaN,NaN,NaN,NaN,NEGOCIACION,NO,SWAP_ICP,C,NEGOCIACION
2,2775,1,FIXED_RATE_MCCY,A,2025-07-10,2025-08-10,2025-08-11,1.452937e+04,163.70,3.190407e+01,...,NaN,NaN,NaN,NaN,NaN,SWAP,NO,SWAP_MONE,C,NEGOCIACION
3,2775,2,OVERNIGHT_INDEX,P,2025-07-10,2025-08-10,2025-08-11,-3.426047e+08,-4693215.00,-2.174303e+06,...,ICPCLP,25447.13,25558.32,4.0,NaN,SWAP,NO,SWAP_MONE,C,NEGOCIACION
4,2938,1,FIXED_RATE_MCCY,A,2025-07-09,2025-08-09,2025-08-11,1.703948e+04,128.27,4.695323e+01,...,NaN,NaN,NaN,NaN,NaN,SWAP,NO,SWAP_MONE,C,NEGOCIACION
5,2938,2,OVERNIGHT_INDEX,P,2025-07-09,2025-08-09,2025-08-11,-4.429637e+08,-3920033.00,-3.200290e+06,...,ICPCLP,25443.60,25558.32,4.0,NaN,SWAP,NO,SWAP_MONE,C,NEGOCIACION
6,3661,1,FIXED_RATE_MCCY,A,2025-02-11,2025-08-11,2025-08-11,1.700000e+09,0.00,2.427411e+07,...,NaN,NaN,NaN,NaN,NaN,SWAP,NO,SWAP_ICP,C,NEGOCIACION
7,3661,2,OVERNIGHT_INDEX_MCCY,P,2025-02-11,2025-08-11,2025-08-11,-1.700000e+09,0.00,-4.437277e+04,...,ICPCLP,24926.00,25558.32,6.0,NaN,SWAP,NO,SWAP_ICP,C,NEGOCIACION
8,4004,1,FIXED_RATE_MCCY,A,2025-02-11,2025-08-11,2025-08-11,3.300000e+09,0.00,6.130621e+07,...,NaN,NaN,NaN,NaN,NaN,SWAP,NO,SWAP_ICP,C,NEGOCIACION
9,4004,2,OVERNIGHT_INDEX_MCCY,P,2025-02-11,2025-08-11,2025-08-11,-3.300000e+09,0.00,-8.613538e+04,...,ICPCLP,24926.00,25558.32,6.0,NaN,SWAP,NO,SWAP_ICP,C,NEGOCIACION


In [14]:
#df_explained_ok.to_csv(f"./{f_proc}_explained_plus_plus.csv")

Ahora vamos a convertir la dataframe en una vista por operación.

In [15]:
import numpy as np

Crear una función helper para agregar los datos correctamente

In [16]:
def aggregate_deal_data(group):
    """
    Aggregate deal data from leg-level to deal-level
    """
    # Get deal-level attributes (should be the same for all legs)
    deal_attrs = {
        'DealNumber': group['dealNumber'].iloc[0],
        'product': group['product'].iloc[0],
        'fechaFinal': group['fechaFinal'].iloc[0],
        'settlementMechanism': group['settlementMechanism'].iloc[0],
        'hedgeAccounting': group['hedgeAccounting'].iloc[0],
        'regulatoryPortfolio': group['regulatoryPortfolio'].iloc[0]
    }
    
    # Separate active (A) and passive (P) legs
    active_legs = group[group['recPay'] == 'A']
    passive_legs = group[group['recPay'] == 'P']
    
    # Process active legs
    if len(active_legs) > 0:
        deal_attrs['pataActivaAmortizacion'] = active_legs['amortizacion'].sum()
        deal_attrs['pataActivaMonedaNocional'] = active_legs['monedaNocional'].iloc[0]  # Assuming same currency for all active legs
        # Get fechaPago from lowest legNumber for active legs
        deal_attrs['pataActivaFechaPagoAmortizacion'] = active_legs.loc[active_legs['legNumber'].idxmin(), 'fechaPago']
        # Sum flujoMonedaPago, handling NaN values
        active_flujo_values = active_legs['flujoMonedaPago'].fillna(active_legs['flujo'])
        deal_attrs['pataActivaFlujo'] = active_flujo_values.sum() if not active_flujo_values.isna().all() else np.nan
    else:
        deal_attrs['pataActivaAmortizacion'] = np.nan
        deal_attrs['pataActivaMonedaNocional'] = np.nan
        deal_attrs['pataActivaFechaPagoAmortizacion'] = np.nan
        deal_attrs['pataActivaFlujo'] = np.nan
    
    # Process passive legs
    if len(passive_legs) > 0:
        deal_attrs['pataPasivaAmortizacion'] = passive_legs['amortizacion'].sum()
        deal_attrs['pataPasivaMonedaNocional'] = passive_legs['monedaNocional'].iloc[0]  # Assuming same currency for all passive legs
        # Get fechaPago from lowest legNumber for passive legs
        deal_attrs['pataPasivaFechaPagoAmortizacion'] = passive_legs.loc[passive_legs['legNumber'].idxmin(), 'fechaPago']
        # Sum flujoMonedaPago, handling NaN values
        passive_flujo_values = passive_legs['flujoMonedaPago'].fillna(passive_legs['flujo'])
        deal_attrs['pataPasivaFlujo'] = passive_flujo_values.sum() if not passive_flujo_values.isna().all() else np.nan
    else:
        deal_attrs['pataPasivaAmortizacion'] = np.nan
        deal_attrs['pataPasivaMonedaNocional'] = np.nan
        deal_attrs['pataPasivaFechaPagoAmortizacion'] = np.nan
        deal_attrs['pataPasivaFlujo'] = np.nan
    
    return pd.Series(deal_attrs)

Agrupamos por deal number

In [17]:
df_deal_level = df_explained_ok.groupby('dealNumber').apply(aggregate_deal_data).reset_index(drop=True)

C:\Users\bencl\AppData\Local\Temp\ipykernel_84996\4184180563.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_deal_level = df_explained_ok.groupby('dealNumber').apply(aggregate_deal_data).reset_index(drop=True)


Aplicar conversiones de nombres de producto

In [18]:
# Apply product name conversions
product_mapping = {
    'SWAP_ICP': 'Swap Promedio Cámara'.encode('utf-8').decode('utf-8'),
    'SWAP_MONE': 'Swap Moneda',
    'SWAP_TASA': 'Swap Tasas'
}
df_deal_level['product'] = df_deal_level['product'].map(product_mapping).fillna(df_deal_level['product'])

Y calcular monto neto a compensar en caso de modalidad compensación

In [19]:
df_deal_level['netoACompensar'] = np.nan
mask_compensation = df_deal_level['settlementMechanism'] == 'C'

df_deal_level.loc[mask_compensation, 'netoACompensar'] = (
    df_deal_level.loc[mask_compensation, 'pataActivaFlujo'].fillna(0) + 
    df_deal_level.loc[mask_compensation, 'pataPasivaFlujo'].fillna(0)
)

Y reordenamos las columnas

In [20]:
# Reorder columns to include the new netoACompensar column
column_order = [
    'DealNumber', 'product', 'fechaFinal', 'settlementMechanism', 'hedgeAccounting', 
    'regulatoryPortfolio', 'pataActivaAmortizacion', 'pataActivaMonedaNocional', 
    'pataActivaFechaPagoAmortizacion', 'pataActivaFlujo', 'pataPasivaAmortizacion',
    'pataPasivaMonedaNocional', 'pataPasivaFechaPagoAmortizacion', 'pataPasivaFlujo',
    'netoACompensar'
]

df_deal_level = df_deal_level[column_order]

Y vemos los resultados:

In [21]:
print("Deal-level dataframe shape:", df_deal_level.shape)
print("\nFirst few rows:")
print(df_deal_level.head())

print("\nDataframe info:")
print(df_deal_level.info())

print("\nSample of deal-level data:")
print(df_deal_level.head(10))

Deal-level dataframe shape: (10, 15)

First few rows:
  DealNumber               product  fechaFinal settlementMechanism  \
0       1634  Swap Promedio Cámara  2025-08-11                   C   
1       2775           Swap Moneda  2025-08-10                   C   
2       2938           Swap Moneda  2025-08-09                   C   
3       3661  Swap Promedio Cámara  2025-08-11                   C   
4       4004  Swap Promedio Cámara  2025-08-11                   C   

  hedgeAccounting regulatoryPortfolio  pataActivaAmortizacion  \
0              NO         NEGOCIACION                    0.00   
1              NO         NEGOCIACION                  163.70   
2              NO         NEGOCIACION                  128.27   
3              NO         NEGOCIACION                    0.00   
4              NO         NEGOCIACION                    0.00   

  pataActivaMonedaNocional pataActivaFechaPagoAmortizacion  pataActivaFlujo  \
0                      CLF                      2025-08

In [22]:
df_deal_level

,DealNumber,product,fechaFinal,settlementMechanism,hedgeAccounting,regulatoryPortfolio,pataActivaAmortizacion,pataActivaMonedaNocional,pataActivaFechaPagoAmortizacion,pataActivaFlujo,pataPasivaAmortizacion,pataPasivaMonedaNocional,pataPasivaFechaPagoAmortizacion,pataPasivaFlujo,netoACompensar
0,1634,Swap Promedio Cámara,2025-08-11,C,NO,NEGOCIACION,0.00,CLF,2025-08-11,1.445572e+08,0.0,CLF,2025-08-11,-2.948804e+03,1.445542e+08
1,2775,Swap Moneda,2025-08-10,C,NO,NEGOCIACION,163.70,CLF,2025-08-11,7.659089e+06,-4693215.0,CLP,2025-08-11,-6.867518e+06,7.915708e+05
2,2938,Swap Moneda,2025-08-09,C,NO,NEGOCIACION,128.27,CLF,2025-08-11,6.857090e+06,-3920033.0,CLP,2025-08-11,-7.120323e+06,-2.632333e+05
3,3661,Swap Promedio Cámara,2025-08-11,C,NO,NEGOCIACION,0.00,CLP,2025-08-11,2.497645e+04,0.0,CLP,2025-08-11,-4.437277e+04,-1.939632e+04
4,4004,Swap Promedio Cámara,2025-08-11,C,NO,NEGOCIACION,0.00,CLP,2025-08-11,6.308002e+04,0.0,CLP,2025-08-11,-8.613538e+04,-2.305536e+04
5,5210,Swap Moneda,2025-08-11,C,NO,COBERTURA_RIESGO_POSICION_BALANCE,0.00,CLF,2025-08-11,4.395486e+04,0.0,CLP,2025-08-11,-1.205505e+05,-7.659563e+04
6,5211,Swap Moneda,2025-08-11,C,NO,COBERTURA_RIESGO_POSICION_BALANCE,0.00,CLF,2025-08-11,2.016465e+04,0.0,CLP,2025-08-11,-5.563869e+04,-3.547404e+04
7,5798,Swap Tasas,2025-08-11,C,NO,NEGOCIACION,0.00,USD,2025-08-11,7.578576e+04,NaN,NaN,NaN,NaN,7.578576e+04
8,5799,Swap Moneda,2025-08-11,C,NO,COBERTURA_RIESGO_POSICION_BALANCE,0.00,CLP,2025-08-11,5.311344e+04,0.0,USD,2025-08-11,-5.499729e+04,-1.883854e+03
9,876,Swap Promedio Cámara,2025-08-11,C,NO,NEGOCIACION,0.00,CLF,2025-08-11,8.436145e+02,0.0,CLF,2025-08-11,-4.265847e+07,-4.265763e+07


In [ ]:
#df_deal_level.to_csv(f"./{f_proc}_deal_level.csv", index=False, encoding='utf-8')
#df_deal_level.to_excel(f"./{f_proc}_deal_level.xlsx", index=False)
df_deal_level.to_excel(rf"C:\Users\bencl\OneDrive - palace.cl\Documents\Palace\Banco Internacional\Treasury PM\06 Pruebas\expiry_complement_{f_proc}.xlsx", index=False)


: 